# Testing LG TV in the living room

### Initial discovery, obtaining passkey

In [5]:

from pywebostv.discovery import *
from pywebostv.connection import *
from pywebostv.controls import *

store = {}

client = WebOSClient('192.168.110.220')
client.connect()
for status in client.register(store):
    if status == WebOSClient.PROMPTED:
        print("Please accept the connect on the TV!")
    elif status == WebOSClient.REGISTERED:
        print("Registration successful!")

print(store)

Please accept the connect on the TV!
Registration successful!
{'client_key': 'd173b1786d56b7eac6dc969395bdbcba'}


### Reconnect

In [4]:
from pywebostv.discovery import *
from pywebostv.connection import *
from pywebostv.controls import *
import json

# Load device configuration
with open('config/devices/lg_tv_living.json', 'r') as f:
    config = json.load(f)

store = {"client_key": config['tv']['client_key']}
print(store)
print(config['tv']['ip_address'])

client = WebOSClient(config['tv']['ip_address'])
client.connect()
for status in client.register(store):
    if status == WebOSClient.PROMPTED:
        print("Please accept the connect on the TV!")
    elif status == WebOSClient.REGISTERED:
        print("Registration successful!")

{'client_key': '7cb66fab44c7d5b37f29e7db4d09e571'}
192.168.110.133
Registration successful!


## App Discovery

The following cells will help you discover all installed apps on your LG TV and their IDs. This is useful for programmatically launching apps via the MQTT bridge.

In [2]:
# Import additional libraries for app discovery
import os
import sys
from datetime import datetime

from tabulate import tabulate


In [12]:
# Initialize the app control
app_control = ApplicationControl(client)

# Get all installed apps
print("Retrieving all installed apps...")
apps = app_control.list_apps()
print(f"Found {len(apps)} apps installed on the TV.")
for app in apps:
    print(app["title"])

Retrieving all installed apps...
Found 115 apps installed on the TV.
AirPlay
Amazon Alexa
сМАРТ МЕСЯЦ
Кинопоиск
Apple TV
KION
Shop on TV
Первое использование
Настройки
LivePlus
LivePlus
LivePlus
LivePlus
LivePlus
LivePlus
Реклама
Ad Overlay App
Ad Overlay App
Ad Overlay App
Ad Overlay App
Ad Overlay App
Ad Overlay App extension
APP CASTING
Line View
BeanBrowser
Веб-браузер
Воспроизведение аудио по Bluetooth
Bluetooth Surround Auto Tuning
Камера
Remote Management for OLED TV
РЕДАКТОР КАНАЛОВ
Поиск каналов
Настройки универсального пульта
Connected Red Button
Поддержка клиентов
Dangbei Full View
Dangbei Assistant
Apps
DvrPopup
User Agreement
AV
AV2
Компонент
AV1
QML Factorywin
GameOptimizer
HDMI 1
HDMI 2
HDMI 3
HDMI 4
Home
Панель управления домом
Галерея
InputCommon
Installation
Live Dmost
Live Ginga App Launcher
Live HbbTV
Live Interactive Content
Guide
LivePick App
Телетрансляция
Magic Number
Media Player
Учетная запись LG
Screen Share
Screen Share
Уведомления
One Touch Sound Tuning
Fir

In [16]:
# Display all apps in a nicely formatted table
app_data = []
for app in sorted(apps, key=lambda x: x["title"].lower()):
    app_data.append([
        app["title"],
        app["id"],
        app["systemApp"],
        app["visible"]
    ])

print(tabulate(app_data, headers=['App Name', 'App ID', 'System App', 'Visible']))

App Name                                         App ID                                           System App    Visible
-----------------------------------------------  -----------------------------------------------  ------------  ---------
Ad Overlay App                                   com.webos.app.adhdmi1                            True          False
Ad Overlay App                                   com.webos.app.adhdmi2                            True          False
Ad Overlay App                                   com.webos.app.adhdmi3                            True          False
Ad Overlay App                                   com.webos.app.adhdmi4                            True          False
Ad Overlay App                                   com.webos.app.adoverlay                          True          False
Ad Overlay App extension                         com.webos.app.adoverlayex                        True          False
AirPlay                                          a

### Filter Non-System Apps

Often, you're only interested in the user-installed apps, not system apps.

In [18]:
# Display only user-installed (non-system) apps
user_apps = [app for app in apps if not app["systemApp"]]
user_app_data = []

for app in sorted(user_apps, key=lambda x: x["title"].lower()):
    user_app_data.append([
        app["title"],
        app["id"]
    ])

print(f"Found {len(user_apps)} user-installed apps:")
print(tabulate(user_app_data, headers=['App Name', 'App ID']))

Found 15 user-installed apps:
App Name                        App ID
------------------------------  --------------------------------------
Apple TV                        com.apple.appletv
KION                            com.huawei.app.webos.mts.tv.production
LG Channels                     mgg4.5.channelplus
Okko: кино, сериалы, спорт, ТВ  yota.play
PREMIER                         tntwebapp
RUTUBE                          rutube
Shop on TV                      com.lgshop.app79
START: Фильмы и сериалы 4К      start
VK Видео                        vkvideo
Wink                            ru.rt.video.app.tv
YouTube                         youtube.leanback.v4
Иви                             ivi
Кинопоиск                       com.685631.3411
Российские программы            russian.programs
сМАРТ МЕСЯЦ                     com.27668.188461


### Save App Data to File

In [ ]:
# Save all app data to a JSON file
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"lg_tv_apps_{timestamp}.json"

with open(filename, "w") as f:
    json.dump(apps, f, indent=2)

print(f"Saved complete app data to: {os.path.abspath(filename)}")

### Test Launching an App

In [ ]:
# Function to launch an app by ID
def launch_app_by_id(app_id):
    # Find the app with the specified ID
    target_app = None
    for app in apps:
        if app["id"] == app_id:
            target_app = app
            break
    
    if target_app:
        print(f"Launching app: {target_app['title']} ({app_id})")
        app_control.launch(target_app)
        return True
    else:
        print(f"App with ID '{app_id}' not found!")
        return False

# Example: Uncomment to launch Netflix (replace with the correct ID if different)
# launch_app_by_id("netflix")

In [ ]:
# Interactive app launcher
# Provide the app ID from the table above
app_id_to_launch = input("Enter the app ID to launch (e.g., netflix): ")
if app_id_to_launch:
    launch_app_by_id(app_id_to_launch)